In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import gc
from scipy import stats

In [2]:
def fuck(xx):
    counts = np.bincount(xx)
    temp=np.argmax(counts)
    if temp ==0 :
        counts[temp]=0
        temp=np.argmax(counts)
    return temp
def mini(xx):
    t=np.argmin(xx)
    tt=xx[t]
    while tt==0:
        xx[t]=9
        t=np.argmin(xx)
        tt=xx[t]
    return tt

In [3]:
def get_base_info(x):
    return [i.split(':')[-1] for i in x.split(' ')]

def get_speed(x):
    return np.array([i.split(',')[0] for i in x], dtype='float16')

def get_eta(x):
    return np.array([i.split(',')[1] for i in x], dtype='float16')

def get_state(x):
    return np.array([int(i.split(',')[2]) for i in x])

def get_cnt(x):
    return np.array([i.split(',')[3] for i in x], dtype='int16')

In [11]:
def gen_feats(path, mode='is_train'):
    df = pd.read_csv(path, sep=';', header=None)
    df['link'] = df[0].apply(lambda x: x.split(' ')[0])
    if mode == 'is_train':
        df['label'] = df[0].apply(lambda x: int(x.split(' ')[1]))
        df['label'] = df['label'].apply(lambda x: 3 if x > 3 else x)
        df['label'] -= 1
        df['current_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[2]))
        df['future_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[3]))
    else:
        df['label'] = -1
        df['current_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[2]))
        df['future_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[3]))

    df['time_diff'] = df['future_slice_id'] - df['current_slice_id']

    df['curr_state'] = df[1].apply(lambda x: x.split(' ')[-1].split(':')[-1])
    df['now_speed'] = df['curr_state'].apply(lambda x: x.split(',')[0])
    df['now_eta'] = df['curr_state'].apply(lambda x: x.split(',')[1])
    df['now_cnt'] = df['curr_state'].apply(lambda x: x.split(',')[3])
    df['now_state'] = df['curr_state'].apply(lambda x: x.split(',')[2])
    del df[0]

    for i in tqdm(range(1, 6)):
        df['his_info'] = df[i].apply(get_base_info)
        if i == 1:
            flg = 'current'
        else:
            flg = f'his_{(6 - i) * 7}'
        df['his_speed'] = df['his_info'].apply(get_speed)
        df[f'{flg}_speed_min'] = df['his_speed'].apply(lambda x: x.min())
        df[f'{flg}_speed_max'] = df['his_speed'].apply(lambda x: x.max())
        df[f'{flg}_speed_mean'] = df['his_speed'].apply(lambda x: x.mean())
        df[f'{flg}_speed_std'] = df['his_speed'].apply(lambda x: x.std())

        df['his_eta'] = df['his_info'].apply(get_eta)
        df[f'{flg}_eta_min'] = df['his_eta'].apply(lambda x: x.min())
        df[f'{flg}_eta_max'] = df['his_eta'].apply(lambda x: x.max())
        df[f'{flg}_eta_mean'] = df['his_eta'].apply(lambda x: x.mean())
        df[f'{flg}_eta_std'] = df['his_eta'].apply(lambda x: x.std())

        df['his_cnt'] = df['his_info'].apply(get_cnt)
        df[f'{flg}_cnt_min'] = df['his_cnt'].apply(lambda x: x.min())
        df[f'{flg}_cnt_max'] = df['his_cnt'].apply(lambda x: x.max())
        df[f'{flg}_cnt_mean'] = df['his_cnt'].apply(lambda x: x.mean())
        df[f'{flg}_cnt_std'] = df['his_cnt'].apply(lambda x: x.std())

        df['his_state'] = df['his_info'].apply(get_state)
        df[f'{flg}_state_zhong'] = df['his_state'].apply(lambda x:fuck(x))
        df[f'{flg}_state_max'] = df['his_state'].apply(lambda x:x.max())
        df[f'{flg}_state_min'] = df['his_state'].apply(lambda x:mini(x))
        #df[f'{flg}_state'] = df['his_state'].apply(lambda x: Counter(x).most_common()[0][0])
        df.drop([i, 'his_info', 'his_speed', 'his_eta', 'his_cnt', 'his_state'], axis=1, inplace=True)
    if mode == 'is_train':
        df.to_csv(f"is_train/{mode}_{path.split('/')[-1]}", index=False)
    else:
        df.to_csv(f"is_test.csv", index=False)

In [5]:
def f1_score_eval(preds, valid_df):
    labels = valid_df.get_label()
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    scores = f1_score(y_true=labels, y_pred=preds, average=None)
    scores = scores[0]*0.2+scores[1]*0.2+scores[2]*0.8
    return 'f1_score', scores, True

In [3]:
    path = 'is_train/is_train_201907'+str(17) +'.txt'
    train = pd.read_csv(path)

In [4]:
y = np.where(((train.future_slice_id > 210)&(train.future_slice_id < 270))|((train.future_slice_id > 510)&(train.future_slice_id < 570)),1,0)

In [6]:
train['is_high'] = y

In [7]:
train

,link,label,current_slice_id,future_slice_id,time_diff,now_speed,now_eta,now_cnt,now_state,current_speed_min,...,his_7_eta_mean,his_7_eta_std,his_7_cnt_min,his_7_cnt_max,his_7_cnt_mean,his_7_cnt_std,his_7_state_zhong,his_7_state_max,his_7_state_min,is_high
0,383172,0,267,284,17,38.1,41.8,7,1,31.406250,...,39.281250,1.390625,3,7,5.2,1.469694,1,1,1,0
1,209892,0,204,222,18,35.3,40.1,2,1,30.906250,...,31.750000,2.636719,1,3,2.0,0.632456,1,1,1,1
2,641047,0,229,236,7,26.0,30.5,1,1,26.000000,...,27.343750,0.706543,3,6,4.8,1.166190,1,1,1,1
3,682831,0,307,314,7,38.2,21.1,18,1,37.312500,...,17.421875,1.552734,10,13,11.6,1.019804,1,2,1,0
4,376640,0,235,248,13,41.3,26.0,1,1,39.500000,...,21.156250,3.425781,2,5,3.4,1.019804,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523513,674887,0,294,318,24,25.3,11.1,33,1,20.203125,...,21.234375,1.689453,16,22,18.2,2.400000,1,1,1,0
523514,569697,0,333,335,2,29.2,24.6,5,1,28.796875,...,33.593750,0.637207,2,5,4.4,1.200000,1,1,1,0
523515,514590,0,613,643,30,13.3,12.9,6,2,13.296875,...,11.000000,2.054688,7,10,8.4,1.200000,2,2,2,0
523516,68894,2,389,396,7,7.0,10.7,13,3,7.000000,...,16.859375,2.886719,8,14,11.6,2.244994,2,2,1,0


In [10]:
for i in tqdm(range(10,31)):
    path = 'is_train/is_train_201907'+str(i) +'.txt'
    train = pd.read_csv(path)
    y = np.where(((train.future_slice_id > 210)&(train.future_slice_id < 270))|((train.future_slice_id > 510)&(train.future_slice_id < 570)),1,0)
    train['is_high'] = y
    save_path = 'is_train_data/is_train_201907'+str(i)+'.txt'
    train.to_csv(save_path, index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [10:18<00:00, 29.46s/it]


In [11]:
for i in tqdm(range(1,10)):
    path = 'is_train/is_train_2019070'+str(i) +'.txt'
    train = pd.read_csv(path)
    y = np.where(((train.future_slice_id > 210)&(train.future_slice_id < 270))|((train.future_slice_id > 510)&(train.future_slice_id < 570)),1,0)
    train['is_high'] = y
    save_path = 'is_train_data/is_train_2019070'+str(i)+'.txt'
    train.to_csv(save_path, index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [04:24<00:00, 29.41s/it]
